In [1]:
import numpy as np
import time
from time import perf_counter as ttt
import random

# allow func library to be imported
import sys
sys.path.append('../')

from func.convert import *
from dtype import Move, Board
import get
import indicator
import prelegal

# import variables
var_folder = 'C:/Users/lenovo/Desktop/Coding/VSC Projects/Ches_Dev/ChessAI_3/vars/'
ini_board_name = 'ini_board.txt'
valid_cells_name = 'valid_cells.txt'
piece_dirs_name = 'piece_dirs.txt'
threats_name = 'threats.txt'

with open(var_folder + ini_board_name, 'r') as f:
    ini_board = eval(f.read())
    ini_board = dict_to_arr(ini_board)

with open(var_folder + valid_cells_name) as f:
    valid_cells = eval(f.read())

with open(var_folder + piece_dirs_name) as f:
    piece_dirs = eval(f.read())

with open(var_folder + threats_name, 'r') as f:
    threats = eval(f.read())

In [2]:
def random_board_full_dict():
    piece_nums = {1:8, 2:2, 3:2, 4:2, 5:1, 6:1}  # number of pieces per side
    board = {}
    for side in [1, -1]:
        for piece_index, piece_num in piece_nums.items():
            # define possible cells for the piece to be in
            if piece_index == 1:
                valid_cells = [31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88]
            else:
                valid_cells = [21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98]
            
            # king cannot be placed next to another king. remove the area around the existing king from valid cells
            if piece_index*side == -6:
                for dir_to_surrounding in [-11, -10, -9, -1, 0, 1, 9, 10, 11]:  # loop through every cell around the king position
                    drop_cell = king_cell + dir_to_surrounding
                    try:
                        valid_cells.remove(drop_cell)
                    except ValueError:
                        pass

            for _ in range(piece_num):
                while True:
                    # select a random cell
                    rand_index = random.randint(0, len(valid_cells)-1)
                    cell = valid_cells[rand_index]

                    if cell not in board:
                        board[cell] = piece_index * side
                        break

            # record the white king position
            if piece_index == 6:
                king_cell = cell

    return board

In [3]:
castle_status = None
arr = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [4, 5, 6, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, -5, 0, 0, 0],
    [0, 0, -3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, -5, 0, 0, 0, 0, 0],
    [0, 0,-6, 0, 0, 0, 0, 0],
])

board = Board.from_arr(ini_board)

print(board)

[[-4 -2 -3 -5 -6 -3 -2 -4]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1]
 [ 4  2  3  5  6  3  2  4]]
dict_values([True, True, True, True])


In [4]:
def legal1(board, move):
    # pseudomove every move
    new_board = board.move_copy(move)
    return indicator.check(new_board, move.side)

def legal2(board, move):
    # if check: pseudomove
    # else: only pseudomove ally pieces which are in slider positions
    if board.check[move.side]:
        new_board = board.move_copy(move)
        return indicator.check(new_board, move.side)
    # elif check if piece is an ally in slider position from king

    king_to_piece = move.start - board.king_pos[move.side]
    for direction in [1, 9, 10, 11]:
        if king_to_piece % direction == 0:
            if king_to_piece // direction in list(range(-7, 8)):
                new_board = board.move_copy(move)
                return indicator.check(new_board, move.side)
    
    return True


def legal3(board, move):
    # if check: pseudomove
    # else: see if the 2nd piece behind ally piece in slider positions is a threat (whether the ally piece is pinned)
    if board.check[move.side]:
        new_board = board.move_copy(move)
        return indicator.check(new_board, move.side)

    king_to_piece = move.start - board.king_pos[move.side]
    for direction in [-11, -10, -9, -1, 1, 9, 10, 11]:
        if king_to_piece % direction == 0:
            if king_to_piece // direction in list(range(1, 8)):
                piece_behind = get.first_in_direction(board, move.start, direction, 6)
                if piece_behind == -move.side * board.slider_threats[direction]:
                    return False
    return True

In [7]:
def gen_random_data():
    board = Board(random_board_full_dict())
    while True:
        random_move_data = gen_random_move_data()
        piece_sign = get.side(board, random_move_data[0])
        if piece_sign != 0:
            break
    
    move = Move(piece_sign, *random_move_data)
    return board, move

def gen_random_move_data():
    r = random.randint
    return valid_cells[r(0, len(valid_cells)-1)], valid_cells[r(0, len(valid_cells)-1)], 0, 0

board, move = gen_random_data()
print(board)
print(move)
print(legal3(board, move))

[[-2  4 -6  0  2  0  0  0]
 [ 0  1 -4 -1 -1  0  0 -1]
 [-1  3  0  0  0  1  0  0]
 [ 0 -2  0  0 -1  1  0  1]
 [-1  1 -3 -1  0  1  0  0]
 [ 0 -1  0  3  0  0  5  4]
 [ 1  0 -5  0 -3  1  0  0]
 [ 0  2  0 -4  6  0  0  0]]
dict_values([True, True, True, True])
Move(1, 74, 42, 0, 0)
True


create a vector to encode information about the board position and legal moves

for legal moves:
    have a 63x64 space to encode every possible simple move (from any start cell to any final cell)
    these can take either values of 1 or 0 (legal/not legal)

    + additional 0/1 spots for castle moves (4 of them)
    + additional 0/1 spots for pawn promotion moves (8cellsx2sidesx4promo)

    train the model at first to select only legal moves
    how to ensure that the model will output only legal moves when inference is done??
     - use softmax to get a confidence level for each move, pick the move with the highest confidence. 
       if not legal, then pick the 2nd highest confidence and so on

Valid cells:
   [21, 22, 23, 24, 25, 26, 27, 28, 
    31, 32, 33, 34, 35, 36, 37, 38, 
    41, 42, 43, 44, 45, 46, 47, 48,
    51, 52, 53, 54, 55, 56, 57, 58, 
    61, 62, 63, 64, 65, 66, 67, 68, 
    71, 72, 73, 74, 75, 76, 77, 78, 
    81, 82, 83, 84, 85, 86, 87, 88, 
    91, 92, 93, 94, 95, 96, 97, 98]